# hello world demo
根据原始数据训练一个能预测image_label的model


In [3]:
import os
import keras as keras
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [4]:
print(tf.__version__)

2.15.0


In [5]:
print(keras.__version__)

2.15.0


# 导入数据集

这个数据集包含 60 000 张训练图像和 10 000 张测试图像，由美国国家标准与技术研究院（National Institute of Standards and Technology，即 MNIST 中的 NIST）在 20 世纪 80 年代收集而成。是深度学习的“Hello World”


In [6]:
mnist = tf.keras.datasets.mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [7]:
print(train_images.shape)
print(type(train_images))

(60000, 28, 28)
<class 'numpy.ndarray'>


In [8]:
print(len(train_labels))
print(train_labels)
print(type(train_labels))

60000
[5 0 4 ... 5 6 8]
<class 'numpy.ndarray'>


In [9]:
print(test_images.shape)

(10000, 28, 28)


In [10]:
print(len(test_labels))
print(test_labels)
# label为0-9的规律数据
print(test_labels.max())
print(test_labels.min())

10000
[7 2 1 ... 4 5 6]
9
0


# 神经网络架构

- 模型包含 2 个 Dense 层，它们都是密集连接（也叫全连接）的神经层。
- 第 2 层（也是最后一层）是一个 10 路 softmax 分类层，它将返回一个由 10 个概率值（总和为 1）组成的数组
  - image_label一共有10种，所以这里是10


In [23]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(512, activation="relu"),
        tf.keras.layers.Dense(10, activation="softmax"),
    ]
)

# 编译

- 编译（compilation）步骤的 3 个参数。

1.  优化器（optimizer）：模型基于训练数据来自我更新的机制，其目的是提高模型性能。
2.  损失函数（loss function）：模型如何衡量在训练数据上的性能，从而引导自己朝着正确的方向前进。
3.  在训练和测试过程中需要监控的指标（metric）：本例只关心精度（accuracy）


In [26]:
model.compile(
    optimizer="rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

# 数据预处理

将数据变换为模型要求的形状，并缩放到所有值都在[0, 1]区间


In [13]:
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype("float32") / 255

In [14]:
print(train_images)
print(train_images.dtype)
print(train_images.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
float32
(60000, 784)


# "Fitting" 开始训练

* 输出中:
* 一个是模型在训练数据上的损失值（loss），
* 另一个是模型在训练数据上的精度（acc）。
* 我们很快就在训练数据上达到了99的精度


In [27]:
model.fit(train_images, train_labels, epochs=6, batch_size=128)

Epoch 1/6
469/469 [==============================] - 2s 4ms/step - loss: 0.2649 - accuracy: 0.9232
Epoch 2/6
469/469 [==============================] - 2s 4ms/step - loss: 0.1079 - accuracy: 0.9681
Epoch 3/6
469/469 [==============================] - 2s 4ms/step - loss: 0.0710 - accuracy: 0.9788
Epoch 4/6
469/469 [==============================] - 3s 5ms/step - loss: 0.0509 - accuracy: 0.9848
Epoch 5/6
469/469 [==============================] - 3s 5ms/step - loss: 0.0384 - accuracy: 0.9886
Epoch 6/6
469/469 [==============================] - 2s 5ms/step - loss: 0.0294 - accuracy: 0.9912


# 利用模型进行预测

预测新图像的类别概率，这些图像(test_images)不属于训练数据

In [16]:
test_digits = test_images[0:5]
predictions = model.predict(test_digits)

1/1 [==============================] - 0s 86ms/step


## 测试预测效果
数组中每个索引为i的数字对应数字图像属于类别i的概率

In [17]:
# 输入5个，且lable有10种，所以输出是5*10
print(predictions.shape)
print(predictions[0])

(5, 10)
[2.9095791e-08 1.0470873e-09 6.5268580e-07 1.1663474e-05 9.3962571e-14
 3.3168011e-09 2.5530780e-12 9.9998689e-01 1.0266694e-07 5.6545070e-07]


In [18]:
# 找出可能性最大的索引,
predictions[0].argmax()

7

In [19]:
# image_lable=7 的概率是99.9%
predictions[0][7]

0.9999869

In [20]:
# 查看原始数据的lable=7，符合预测结果
test_labels[0]

7

# 用新数据评估模型

测试精度约为98.05%，比训练精度（99.1%）低不少

In [21]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"test_acc: {test_acc}")

313/313 [==============================] - 1s 2ms/step - loss: 0.0617 - accuracy: 0.9806
test_acc: 0.9805999994277954
